<a href="https://colab.research.google.com/github/Yunpei24/BigDataBase/blob/main/Activite2_Question4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ACTIVITE2

Installation du JDK

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Création de la variable d'environnement <JAVA_HOME> pour situer l'emplacement d'installationde Java 

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

# Installation du framework Hadoop

Téléchargement depuis les archives de la fondation Apache

In [ ]:
!wget https://archive.apache.org/dist/hadoop/common/hadoop-3.3.0/hadoop-3.3.0.tar.gz

Extraction de l'archive

In [ ]:
!tar -xzvf hadoop-3.3.0.tar.gz

Copie du dossier extrait dans l'emplacement <user/local>

In [5]:
!cp -r hadoop-3.3.0/ /usr/local/

## Programmation de tâches distribuées avec <i>MapReduce</i>

Création d'un repertoire <myinput> pour contenir le jeu de données à tester durant cet exercice e d'un second pour les résultats du traitement distribué

In [6]:
!mkdir -p ~/myinput
!mkdir -p ~/myoutput

Télachargement du jeu de données dans le fichier <u>purchases.txt</u>

In [ ]:
!curl -L -o 'purchases.txt' 'https://drive.google.com/u/0/uc?id=1NS-PSXW8bSNpzFH4XRbtmMnMGhXBdYy6&export=download&confirm=t'

Déplacement du fichier <u>purchases.txt</u> dans le répertoire <myinput>

In [8]:
!mv purchases.txt ~/myinput/purchases.txt

Vérification que les fichiers ont été bien copiés

In [9]:
!ls /usr/myinput

ls: cannot access '/usr/myinput': No such file or directory


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Affichage des premiers lignes du fichier. Le format des enregistrement est le suivant:
<table border='1'><tr>
<td>Date</td><td>Heure</td><td>Magasin</td><td>Produit</td><td>Montant</td><td>Moyen_de_paiement</td>
</tr></table>
La tabulation <b>\t</b> est utilisée comme séparateur de colonne ✅

In [12]:
!head -5  ~/myinput/purchases.txt

2012-01-01	09:00	San Jose	Men's Clothing	214.05	Amex
2012-01-01	09:00	Fort Worth	Women's Clothing	153.57	Visa
2012-01-01	09:00	San Diego	Music	66.08	Cash
2012-01-01	09:00	Pittsburgh	Pet Supplies	493.51	Discover
2012-01-01	09:00	Omaha	Children's Clothing	235.63	MasterCard


#QUESTION4 : Le nombre total des ventes et la valeur totale des ventes de tous magasins confondus

In [13]:
#!/usr/bin/python
import sys
# Mettez vos instructions ...
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) == 6:
    date, time, store, item, amount, payment = data
    print(store, "\t1\t", amount)

In [14]:
# All cell codes are stored in a List variable "In"
with open('/content/mapper.py', 'w') as f:
  f.write(In[13]) 
f.close()

In [15]:
!chmod u+rwx /content/mapper.py

In [ ]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper.py

In [28]:
#!/usr/bin/python
from operator import itemgetter
import sys
# Mettez vos instructions ...
salesTotalVente = 0
salesTotalAmount = 0
oldKey = None

print("Magazins\tNbresVentes\tChiffreAffaire")
for line in sys.stdin:
  data = line.strip().split("\t")
  if len(data) != 3:
    continue
  
  thisKey, thisVente, thisAmount = data
  if oldKey and oldKey != thisKey:
    print(oldKey, "\t", str(salesTotalVente), "\t", str(salesTotalAmount))
    salesTotalVente = 0
    
  oldKey = thisKey
  salesTotalVente += int (thisVente)
  salesTotalAmount += float(thisAmount) 

if oldKey != None:
  print(oldKey, "\t", str(salesTotalVente), "\t", str(salesTotalAmount))

Magazins	NbresVentes	ChiffreAffaire


In [29]:
# All cell codes are stored in a List variable "In"
with open('/content/reducer.py', 'w') as f:
  f.write(In[28]) 
f.close()

In [30]:
!chmod u+rwx /content/reducer.py

In [36]:
!head -10 ~/myinput/purchases.txt | python3 /content/mapper.py | sort | python3 /content/reducer.py

Magazins	NbresVentes	ChiffreAffaire
Austin  	 1 	 379.6
Corpus Christi  	 1 	 404.98
Fort Worth  	 2 	 772.43
New York  	 1 	 1069.23
Omaha  	 1 	 1304.8600000000001
Pittsburgh  	 1 	 1798.3700000000001
San Diego  	 1 	 1864.45
San Jose  	 1 	 2078.5
Stockton  	 1 	 2325.68


In [ ]:
!rm -r ~/myoutput
!/usr/local/hadoop-3.3.0/bin/hadoop jar /usr/local/hadoop-3.3.0/share/hadoop/tools/lib/hadoop-streaming-3.3.0.jar -input ~/myinput -output ~/myoutput -file /content/mapper.py  -file /content/reducer.py  -mapper 'python mapper.py'  -reducer 'python reducer.py'

In [33]:
!ls ~/myoutput

part-00000  _SUCCESS


In [38]:
!head -n 30 ~/myoutput/part-00000

Magazins	NbresVentes	ChiffreAffaire
Albuquerque  	 40345 	 10052311.42000001
Anaheim  	 40086 	 20128727.780000057
Anchorage  	 39806 	 30062228.179999754
Arlington  	 40348 	 40134436.150000244
Atlanta  	 40168 	 50131582.84999982
Aurora  	 39808 	 60124553.77000028
Austin  	 40332 	 70181712.67000116
Bakersfield  	 40326 	 80212921.59000231
Baltimore  	 40196 	 90309443.04000184
Baton Rouge  	 40387 	 100440716.27000271
Birmingham  	 40253 	 110517322.79000346
Boise  	 40203 	 120556489.53000313
Boston  	 40338 	 130595962.81000313
Buffalo  	 40053 	 140597904.00000566
Chandler  	 39826 	 150517463.86000377
Charlotte  	 40509 	 160629995.20000333
Chesapeake  	 40288 	 170668500.12000126
Chicago  	 40220 	 180731022.19000298
Chula Vista  	 40080 	 190705973.5300034
Cincinnati  	 40452 	 200845479.2700075
Cleveland  	 40313 	 210913315.11000627
Colorado Springs  	 40389 	 220974420.98000512
Columbus  	 40225 	 231009662.01000178
Corpus Christi  	 39925 	 240986184.78000286
Dallas  	 40